<a href="https://colab.research.google.com/github/rafpystarter/Data-Science-Starter/blob/master/Project_4_fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:

import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.7.0


In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-01-31 07:34:15--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2022-01-31 07:34:16 (9.25 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-01-31 07:34:16--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
df_train = pd.read_table(train_file_path, header = None, names = ['Category','Message'])
df_train.head()

,Category,Message
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
df_test = pd.read_table(test_file_path, header = None, names = ['Category','Message'])
df_test.head()

,Category,Message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [ ]:
#df_train.groupby('Category').describe()

In [ ]:
# downsampling
# df_spam = df_train[df_train['Category']=='spam']
# df_ham = df_train[df_train['Category']=='ham']

In [ ]:
# df_ham_downsample = df_ham.sample(560)

In [ ]:
# df_balanced = pd.concat([df_spam, df_ham_downsample])

In [ ]:
# df_balanced.groupby('Category').describe()

In [ ]:
df_train['spam'] = df_train['Category'].apply(lambda x: 1 if x == 'spam' else 0)

In [ ]:
df_train.tail()

,Category,Message,spam
4174,ham,just woke up. yeesh its late. but i didn't fal...,0
4175,ham,what do u reckon as need 2 arrange transport i...,0
4176,spam,free entry into our £250 weekly competition ju...,1
4177,spam,-pls stop bootydelious (32/f) is inviting you ...,1
4178,ham,tell my bad character which u dnt lik in me. ...,0


In [ ]:
x_train = df_train['Message']

In [ ]:
x_label = df_train['spam']

In [ ]:
df_test.head()

,Category,Message
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [ ]:
df_test['spam'] = df_test['Category'].apply(lambda x: 1 if x == 'spam' else 0)

In [ ]:
y_test = df_test['Message']

In [ ]:
y_label = df_test['spam']

In [ ]:
import tensorflow_hub as hub
!pip install tensorflow-text

     |████████████████████████████████| 4.9 MB 5.2 MB/s 


In [ ]:
import tensorflow_text as text

In [ ]:
bert_preprocess = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_encoder = hub.KerasLayer('https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4')

In [ ]:
input_text = tf.keras.layers.Input(shape=(),dtype=tf.string,name='text')
preprocessed_text = bert_preprocess(input_text)
output = bert_encoder(preprocessed_text)

In [ ]:
layer = tf.keras.layers.Dropout(0.1, name = 'dropout')(output['pooled_output'])
layer = tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'output')(layer)

In [ ]:
model = tf.keras.Model(inputs = [input_text], outputs = [layer])

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [ ]:
model.fit(x_train, x_label, epochs = 10)

Epoch 1/10
131/131 [==============================] - 1827s 14s/step - loss: 0.3833 - accuracy: 0.8457
Epoch 2/10
131/131 [==============================] - 1802s 14s/step - loss: 0.2624 - accuracy: 0.8789
Epoch 3/10
131/131 [==============================] - 1804s 14s/step - loss: 0.2126 - accuracy: 0.9074
Epoch 4/10
131/131 [==============================] - 1796s 14s/step - loss: 0.1864 - accuracy: 0.9277
Epoch 5/10
131/131 [==============================] - 1793s 14s/step - loss: 0.1691 - accuracy: 0.9383
Epoch 6/10
131/131 [==============================] - 1808s 14s/step - loss: 0.1529 - accuracy: 0.9469
Epoch 7/10
131/131 [==============================] - 1813s 14s/step - loss: 0.1427 - accuracy: 0.9533
Epoch 8/10
131/131 [==============================] - 1799s 14s/step - loss: 0.1326 - accuracy: 0.9553
Epoch 9/10
131/131 [==============================] - 1786s 14s/step - loss: 0.1264 - accuracy: 0.9596
Epoch 10/10
131/131 [==============================] - 1789s 14s/step - l

In [ ]:
model.evaluate(y_test,y_label)

44/44 [==============================] - 615s 14s/step - loss: 0.1134 - accuracy: 0.9619


[0.11344177275896072, 0.9619252681732178]

In [ ]:
model.predict(['how are you doing today'])[0][0]

0.0022428036

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred = model.predict([pred_text])
  p = pred[0][0]
  return [p, 'ham' if p<0.05 else 'spam']

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.0049582124, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    print(prediction)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


[0.0022428036, 'ham']
[0.2397561, 'spam']
[0.008554995, 'ham']
[0.41328046, 'spam']
[0.086936414, 'spam']
[0.004538268, 'ham']
[0.007820517, 'ham']
You passed the challenge. Great job!
